In [1]:
# import os, sys, math
import pandas as pd
import re

In [2]:
# the function return amino acid with 1 beta Chain only 
def aa_one_beta_chain(D):
    # Take TRB chain out aa column 
    D['cell_clono_cdr3_aa_TRB'] = D.apply(lambda x : re.findall('TRB:.*', x['cell_clono_cdr3_aa']),axis=1)
    D['cell_clono_cdr3_aa_TRB_count'] = D.apply(lambda x : x['cell_clono_cdr3_aa'].count('TRB'),axis=1)
    D['cell_clono_cdr3_aa_TRB_count'] = D.apply(lambda x : x['cell_clono_cdr3_aa'].count('TRB'),axis=1)
    
    
    # take out the cells with one TRB  
    D = D[D['cell_clono_cdr3_aa_TRB_count'] == 1]
    
    
    # remove the string "TRB:", "[" and "]"
    D['aa_1TRB_only'] = D.apply(lambda x : str(x['cell_clono_cdr3_aa_TRB']).replace('TRB:',''),axis=1)
    D['aa_1TRB_only'] = D.apply(lambda x : str(x['aa_1TRB_only']).replace('[',''),axis=1)
    D['aa_1TRB_only'] = D.apply(lambda x : str(x['aa_1TRB_only']).replace(']',''),axis=1)
    D['aa_1TRB_only'] = D.apply(lambda x : str(x['aa_1TRB_only']).strip('\''),axis=1)
    

    return D

In [3]:
def max_true_binder (all_D_binder, num_virus_with_most_binding = 1):


    # show the binder percentage 
    from numpy import empty
    binder_count = empty([len(all_D_binder.columns),2])

    for k in range(0,len(all_D_binder.columns)):
        binder_count[k][0]= (all_D_binder[all_D_binder.columns[k]]).sum()
        binder_count[k][1]= (~all_D_binder[all_D_binder.columns[k]]).sum()
    
    

    sorted(range(len(binder_count[:,:1])), key=lambda x: binder_count[x,:1])
    max_loc = sorted(range(len(binder_count[:,:1])), key=lambda x: binder_count[x,:1])[-num_virus_with_most_binding:]
    
    all_D_binder.iloc[:,max_loc]

    return all_D_binder.iloc[:,max_loc]

In [4]:
def extra_raw_data(): 
    # import raw data 
    D1 = pd.read_csv("../CD8+TCellsDataset/vdj_v1_hs_aggregated_donor1_binarized_matrix.csv")
    D2 = pd.read_csv("../CD8+TCellsDataset/vdj_v1_hs_aggregated_donor2_binarized_matrix.csv")
    D3 = pd.read_csv("../CD8+TCellsDataset/vdj_v1_hs_aggregated_donor3_binarized_matrix.csv")
    D4 = pd.read_csv("../CD8+TCellsDataset/vdj_v1_hs_aggregated_donor4_binarized_matrix.csv")
    
    # extract amino acid sequences with 1 beta chain only 
    D1_aa_1beta = aa_one_beta_chain(D1)
    D2_aa_1beta = aa_one_beta_chain(D2)
    D3_aa_1beta = aa_one_beta_chain(D3)
    D4_aa_1beta = aa_one_beta_chain(D4)
    
    # combine all the data from 4 donors into one big table 
    all_D = pd.concat([D1_aa_1beta, D2_aa_1beta, D3_aa_1beta, D4_aa_1beta], ignore_index=True, sort=False)
    
    table = all_D.select_dtypes('bool').join([all_D['aa_1TRB_only']])
    
    return table

In [5]:
dt = extra_raw_data()

/Users/Pi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/Pi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/Users/Pi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

In [6]:
# this function will return the virus antigen which the most binding cases 
# you can choose num_of_virus for any other number of virus antigens that you want 

def virus_with_most_true_cases (all_D_binder, num_of_virus = 1):

    from numpy import empty
    binder_count = empty([len(all_D_binder.columns),2])

    for k in range(0,len(all_D_binder.columns)):
        binder_count[k][0]= (all_D_binder[all_D_binder.columns[k]]).sum()
        binder_count[k][1]= (~all_D_binder[all_D_binder.columns[k]]).sum()
        
    sorted(range(len(binder_count)), key=lambda x: binder_count[x][0])
    max_loc = sorted(range(len(binder_count)), key=lambda x: binder_count[x][0])[-num_of_virus:]
    
    all_D_binder.iloc[:,max_loc]
    
    return all_D_binder.iloc[:,max_loc]

In [7]:
def generate_balanced_aa_antigen_table(dt): 
    
    # pick the virus with the most binding cases
    virus_with_most_binding =  virus_with_most_true_cases(dt.select_dtypes('bool'), num_of_virus = 1)
    
    # combine the amino-acid sequence and the particular virus antigen that you are interested 
    aa_antigen_table = dt[list(virus_with_most_binding)].join([dt['aa_1TRB_only']])
    
    
    # find the antigen with the most binding cases 
#     print(aa_antigen_table.columns[0].replace('_binder', '') + ' has the most true binding cases') 
#     aa_antigen_table[aa_antigen_table.columns[0]].value_counts().sort_index().plot.bar()

    # since the data is unbalanced, we need to balance the data by randomly selecting the same amount of true cases from the false cases
    aa_antigen_table_false = aa_antigen_table[aa_antigen_table[aa_antigen_table.columns[0]] == False]
    aa_antigen_table_true = aa_antigen_table[aa_antigen_table[aa_antigen_table.columns[0]] == True]
    aa_antigen_table_false_random_sample = aa_antigen_table_false.sample(n = len(aa_antigen_table_true))
    balanced_aa_antigen_table = pd.concat([aa_antigen_table_false_random_sample, aa_antigen_table_true], ignore_index=True, sort=False)

#     print(balanced_aa_antigen_table.columns[0].replace('_binder', '') + ' has the most binding cases and is balanced as shown below') 
# #     balanced_aa_antigen_table[balanced_aa_antigen_table.columns[0]].value_counts().sort_index().plot.bar()
    
    # re-label 
    balanced_aa_antigen_table["classes"] = balanced_aa_antigen_table[balanced_aa_antigen_table.columns[0]].astype(int)
    balanced_aa_antigen_table = balanced_aa_antigen_table[{'aa_1TRB_only', 'classes'}]
    
    return balanced_aa_antigen_table
    

In [15]:
balanced_aa_antigen_table = generate_balanced_aa_antigen_table(dt)